In [32]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate
def get(q):
    return pd.DataFrame(sql.execute(q))

In [33]:
#sanity check on date matching 
#might adjust with three months in future
q='''select project_id, min(publish_date),max(publish_date), start_date, end_date
from cve c
join project p on c.project_id = p.id
group by project_id, start_date, end_date;'''

In [34]:
q='''select name, count(*) as total
from cve c
join project p on c.project_id = p.id
group by name;'''
tc=get(q)
tc

,name,total
0,Firefox,1731
1,Thunderbird,940
2,Linux,3144
3,Samba,138
4,LibreOffice,40
5,OpenCV,33
6,Kodi,3


In [35]:
q='''select name, count(*) as memory
from cve c
join project p on c.project_id = p.id
join memory_cwe mc on c.CWE = mc.CWE
where memory=1
group by name;'''
mc=get(q)
mc

,name,memory
0,Firefox,624
1,Thunderbird,426
2,Linux,1427
3,Samba,41
4,LibreOffice,16
5,OpenCV,25


In [39]:
df=reduce(lambda x,y: pd.merge(x,y,on='name', how='left'),[tc,mc])
df=df.fillna(0)
df=df.sort_values('total',ascending=False)
df.memory=df.memory.astype(int)
df.memory = round((df.memory / df.total)*100,1)


In [40]:
df.memory.median()

40.0

In [37]:
df.total=df.total.apply(lambda x: '{:,}'.format(x))
df.memory=df.memory.astype(str) + '%'
df=df.rename(columns={'name':'Project','total':'No. of CVEs (within study timeline)','memory':'memory-related CVEs'})
print(tabulate(df, tablefmt='pipe', headers='keys',showindex=False))

| Project     | No. of CVEs (within study timeline)   | memory-related CVEs   |
|:------------|:--------------------------------------|:----------------------|
| Linux       | 3,144                                 | 45.4%                 |
| Firefox     | 1,731                                 | 36.0%                 |
| Thunderbird | 940                                   | 45.3%                 |
| Samba       | 138                                   | 29.7%                 |
| LibreOffice | 40                                    | 40.0%                 |
| OpenCV      | 33                                    | 75.8%                 |
| Kodi        | 3                                     | 0.0%                  |
